![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Working Memory with Long-Term Extraction Strategies

## Introduction

This notebook demonstrates how to implement **working memory** with configurable **long-term extraction strategies** that inform memory management tools about when and how to extract important information from working memory to long-term storage.

### Key Concepts

- **Working Memory**: Persistent storage for task-focused context (conversation messages, task-related data)
- **Long-term Memory**: Cross-session knowledge (user preferences, important facts learned over time)
- **Long-Term Extraction Strategy**: Configurable logic for when/how to move important information from working to long-term memory
- **Strategy-Aware Tools**: Memory tools that understand the extraction strategy and make intelligent decisions
- **Context-Informed LLM**: The LLM receives information about the extraction strategy to make better memory management decisions

### The Problem We're Solving

Previously, memory tools like `add_memories_to_working_memory` and `create_memory` operated without knowledge of:
- When memories should be extracted from working memory
- What criteria determine memory importance
- How the working memory's extraction strategy affects tool behavior

This notebook shows how to solve this by making tools **extraction strategy aware**.

In [ ]:
# Install the Redis Context Course package
import subprocess
import sys
import os

# Install the package in development mode
package_path = "../../reference-agent"
result = subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-e", package_path], 
                      capture_output=True, text=True)
if result.returncode == 0:
    print("✅ Package installed successfully")
else:
    print(f"❌ Package installation failed: {result.stderr}")
    raise RuntimeError(f"Failed to install package: {result.stderr}")

In [ ]:
import os
import sys

# Set up environment - handle both interactive and CI environments
def _set_env(key: str):
    if key not in os.environ:
        # Check if we're in an interactive environment
        if hasattr(sys.stdin, 'isatty') and sys.stdin.isatty():
            import getpass
            os.environ[key] = getpass.getpass(f"{key}: ")
        else:
            # Non-interactive environment (like CI) - use a dummy key
            print(f"⚠️  Non-interactive environment detected. Using dummy {key} for demonstration.")
            os.environ[key] = "sk-dummy-key-for-testing-purposes-only"

_set_env("OPENAI_API_KEY")

# Set Redis URL
os.environ["REDIS_URL"] = "redis://localhost:6379"

## Working Memory Components

Let's explore the key components of our working memory system:

In [ ]:
# Import working memory components
from redis_context_course.working_memory import (
    WorkingMemory, 
    MessageCountStrategy, 
    LongTermExtractionStrategy,
    WorkingMemoryItem
)
from redis_context_course.working_memory_tools import WorkingMemoryToolProvider
from redis_context_course.memory_client import MemoryClient
from langchain_core.messages import HumanMessage, AIMessage

print("✅ Working memory components imported successfully")

## 1. Long-Term Extraction Strategies

Extraction strategies define **when** and **how** memories should be moved from working memory to long-term storage:

In [ ]:
# Create different extraction strategies
print("🎯 Available Extraction Strategies")
print("=" * 50)

# Strategy 1: Message Count Strategy
strategy1 = MessageCountStrategy(message_threshold=5, min_importance=0.6)
print(f"📊 Strategy: {strategy1.name}")
print(f"   Trigger: {strategy1.trigger_condition}")
print(f"   Priority: {strategy1.priority_criteria}")
print(f"   Config: {strategy1.config}")

# Strategy 2: More aggressive extraction
strategy2 = MessageCountStrategy(message_threshold=3, min_importance=0.4)
print(f"\n📊 Strategy: {strategy2.name} (Aggressive)")
print(f"   Trigger: {strategy2.trigger_condition}")
print(f"   Priority: {strategy2.priority_criteria}")
print(f"   Config: {strategy2.config}")

# Demonstrate importance calculation
print("\n🧮 Importance Calculation Examples:")
test_contents = [
    "I prefer online courses",
    "My goal is to become a data scientist",
    "What time is it?",
    "I love machine learning and want to specialize in it",
    "The weather is nice today"
]

for content in test_contents:
    importance = strategy1.calculate_importance(content, {})
    print(f"   '{content}' → importance: {importance:.2f}")

## 2. Working Memory in Action

Let's see how working memory operates with an extraction strategy:

In [ ]:
# Initialize working memory with strategy
student_id = "demo_student_working_memory"
strategy = MessageCountStrategy(message_threshold=4, min_importance=0.5)

# Note: This will fail if Redis is not available, which is expected in some environments
try:
    working_memory = WorkingMemory(student_id, strategy)
    memory_client = MemoryClient(student_id)
    
    print("✅ Working memory initialized successfully")
    print(f"📊 Strategy: {working_memory.extraction_strategy.name}")
    print(f"📊 Trigger: {working_memory.extraction_strategy.trigger_condition}")
    
    redis_available = True
except Exception as e:
    print(f"⚠️  Redis not available: {e}")
    print("📝 Continuing with conceptual demonstration...")
    redis_available = False

In [ ]:
if redis_available:
    # Simulate a conversation
    print("💬 Simulating Conversation")
    print("=" * 40)
    
    messages = [
        HumanMessage(content="I prefer online courses because I work part-time"),
        AIMessage(content="I understand you prefer online courses due to your work schedule."),
        HumanMessage(content="My goal is to specialize in machine learning"),
        AIMessage(content="Machine learning is an excellent specialization!"),
        HumanMessage(content="What courses do you recommend?"),
    ]
    
    for i, message in enumerate(messages, 1):
        working_memory.add_message(message)
        msg_type = "👤 Human" if isinstance(message, HumanMessage) else "🤖 AI"
        print(f"{i}. {msg_type}: {message.content}")
        print(f"   Working memory size: {len(working_memory.items)}")
        print(f"   Should extract: {working_memory.should_extract_to_long_term()}")
        
        if working_memory.should_extract_to_long_term():
            print("   🔄 EXTRACTION TRIGGERED!")
            break
        print()
    
    # Show working memory contents
    print("\n📋 Working Memory Contents:")
    for i, item in enumerate(working_memory.items, 1):
        print(f"{i}. [{item.message_type}] {item.content[:50]}... (importance: {item.importance:.2f})")
else:
    print("📝 Conceptual demonstration of working memory behavior:")
    print("")
    print("1. 👤 Human: I prefer online courses because I work part-time")
    print("   Working memory size: 1, Should extract: False")
    print("")
    print("2. 🤖 AI: I understand you prefer online courses due to your work schedule.")
    print("   Working memory size: 2, Should extract: False")
    print("")
    print("3. 👤 Human: My goal is to specialize in machine learning")
    print("   Working memory size: 3, Should extract: False")
    print("")
    print("4. 🤖 AI: Machine learning is an excellent specialization!")
    print("   Working memory size: 4, Should extract: True")
    print("   🔄 EXTRACTION TRIGGERED!")

## 3. Strategy-Aware Memory Tools

The key innovation is that memory tools now have access to the working memory's extraction strategy configuration:

In [ ]:
if redis_available:
    # Create strategy-aware tools
    tool_provider = WorkingMemoryToolProvider(working_memory, memory_manager)
    tools = tool_provider.get_memory_tool_schemas()
    
    print("🛠️  Strategy-Aware Memory Tools")
    print("=" * 50)
    
    for tool in tools:
        print(f"📋 {tool.name}")
        print(f"   Description: {tool.description.split('.')[0]}...")
        print()
    
    # Show the strategy context that gets injected into tool descriptions
    print("🎯 Strategy Context for Tools:")
    print("-" * 30)
    context = tool_provider.get_strategy_context_for_system_prompt()
    print(context)
else:
    print("🛠️  Strategy-Aware Memory Tools (Conceptual)")
    print("=" * 50)
    print("📋 add_memories_to_working_memory")
    print("   - Knows current extraction strategy")
    print("   - Understands when extraction will trigger")
    print("   - Can make intelligent decisions about memory placement")
    print()
    print("📋 create_memory")
    print("   - Uses strategy to calculate importance")
    print("   - Decides between working memory vs direct long-term storage")
    print("   - Considers extraction strategy in decision making")
    print()
    print("📋 get_working_memory_status")
    print("   - Provides full context about current strategy")
    print("   - Shows extraction readiness")
    print("   - Helps LLM make informed decisions")

## 4. Tool Descriptions with Strategy Context

Let's examine how the extraction strategy context is embedded in tool descriptions:

In [ ]:
if redis_available:
    # Show how strategy context is embedded in tool descriptions
    print("📝 Example Tool Description with Strategy Context")
    print("=" * 60)
    
    create_memory_tool = next(tool for tool in tools if tool.name == "create_memory")
    print(f"Tool: {create_memory_tool.name}")
    print(f"Description:")
    print(create_memory_tool.description)
else:
    print("📝 Example Tool Description with Strategy Context (Conceptual)")
    print("=" * 60)
    print("Tool: create_memory")
    print("Description:")
    print("""
Create a memory with extraction strategy awareness.

This tool creates a memory and decides whether to store it immediately in
long-term storage or add it to working memory based on the extraction strategy.

WORKING MEMORY CONTEXT:
- Current extraction strategy: message_count
- Extraction trigger: After 4 messages
- Priority criteria: Items with importance >= 0.5, plus conversation summary
- Current working memory size: 4 items
- Last extraction: Never
- Should extract now: True

This context should inform your decisions about when and what to store in memory.
""")

## 5. Integration with Agent System

The working memory system integrates seamlessly with the ClassAgent:

In [ ]:
if redis_available:
    # Demonstrate agent integration
    from redis_context_course import ClassAgent
    
    print("🤖 Agent Integration with Working Memory")
    print("=" * 50)
    
    try:
        # Initialize agent with working memory
        agent = ClassAgent("demo_student_agent", extraction_strategy="message_count")
        
        print("✅ Agent initialized with working memory")
        print(f"📊 Working memory strategy: {agent.working_memory.extraction_strategy.name}")
        print(f"📊 Available tools: {len(agent._build_graph().get_graph().nodes)} nodes in workflow")
        
        # Show that the agent has working memory tools
        base_tools = [
            agent._search_courses_tool,
            agent._get_recommendations_tool,
            agent._store_preference_tool,
            agent._store_goal_tool,
            agent._get_student_context_tool
        ]
        working_memory_tools = agent.working_memory_tools.get_memory_tool_schemas()
        
        print(f"📋 Base tools: {len(base_tools)}")
        print(f"📋 Working memory tools: {len(working_memory_tools)}")
        print(f"📋 Total tools available to LLM: {len(base_tools + working_memory_tools)}")
        
        print("\n🎯 Working Memory Tools Available to Agent:")
        for tool in working_memory_tools:
            print(f"   - {tool.name}")
            
    except Exception as e:
        print(f"⚠️  Agent initialization failed: {e}")
        print("This is expected if OpenAI API key is not valid")
else:
    print("🤖 Agent Integration with Working Memory (Conceptual)")
    print("=" * 50)
    print("✅ Agent can be initialized with working memory extraction strategy")
    print("📊 Working memory tools are automatically added to agent's toolkit")
    print("📊 System prompt includes working memory strategy context")
    print("📊 Messages are automatically added to working memory")
    print("📊 Extraction happens automatically based on strategy")

## Key Benefits

### ✅ **Strategy Awareness**
- Memory tools understand the current extraction strategy
- Tools can make intelligent decisions about memory placement
- LLM receives context about when extraction will happen

### ✅ **Intelligent Memory Management**
- High-importance memories can bypass working memory
- Extraction happens automatically based on configurable triggers
- Memory tools coordinate with extraction strategy

### ✅ **Configurable Behavior**
- Different extraction strategies for different use cases
- Importance calculation can be customized
- Trigger conditions are flexible and extensible

### ✅ **Context-Informed Decisions**
- Tools include strategy context in their descriptions
- LLM can make better decisions about memory management
- System prompt includes working memory status

## Next Steps

This working memory system with extraction strategy awareness provides a foundation for:

1. **Custom Extraction Strategies**: Implement time-based, importance-threshold, or conversation-end strategies
2. **Advanced Importance Calculation**: Use NLP techniques for better importance scoring
3. **Multi-Modal Memory**: Extend to handle different types of content (text, images, etc.)
4. **Memory Hierarchies**: Implement multiple levels of memory with different retention policies

The key insight is that **memory tools should be aware of the memory management strategy** to make intelligent decisions about when and how to store information.